# WP1 - Anomaly subgraph extraction: Finding Granger causalities

In [2]:
import py2neo
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

In [2]:
# TODO: remove before upload
graph = py2neo.Graph("bolt://192.168.1.185:7687", auth=("neo4j", ""))

In [3]:
graph_rel = graph.relationships.match().all()

### Calculate granger causality on dataset and update graph

In [3]:
MAXLAG = 2
f = '../data/loadtest-Updated.csv'
P = pd.read_csv(f).sort_values(by=['Time', 'Unnamed: 0'])

In [4]:
#perform Granger-Causality test
for rel in graph_rel:
    source = rel.start_node['id']
    target = rel.end_node['id']
    if source in P.columns and target in P.columns:
        gc_dicts = grangercausalitytests(P[[source, target]], MAXLAG, verbose=False)
        p_values = [lag_result[0]['ssr_chi2test'][1] for lag_result in gc_dicts.values()]
        print(source, target)
        if np.min(p_values) < 0.05:
            print('causal')
            rel["causal"] = True
            graph.push(rel)
        else:
            print('not causal')
            rel["causal"] = False
            graph.push(rel)
           

carts front-end
not causal
user user-db
causal
user front-end
not causal
catalogue front-end
not causal
catalogue-db catalogue
not causal
orders-db orders
causal
carts-db carts
causal
orders shipping
causal
orders front-end
not causal
shipping queue-master
causal


In [15]:
neighbor_q1 = '''MATCH (a)-[r]-(b) where a.id='worker1' RETURN b'''
worker1_neighbors = [i.get('b') for i in graph.run(neighbor_q1).data()]

In [16]:
neighbor_q2 = '''MATCH (a)-[r]-(b) where a.id='worker2' RETURN b'''
worker2_neighbors = [i.get('b') for i in graph.run(neighbor_q2).data()]

In [18]:
def granger_causality_for_neighbors(neighbors, graph):
    for i in range(len(neighbors)):
        for j in range(i+1, len(neighbors)):
            n1 = neighbors[i]
            n2 = neighbors[j]
            node1 = n1['id']
            node2 = n2['id']
            if node1 == node2:
                continue
            if node1 in P.columns and node2 in P.columns:
                gc_dicts_direction_1 = grangercausalitytests(P[[node1, node2]], MAXLAG, verbose=False)
                p_values_direction_1 = [lag_result[0]['ssr_chi2test'][1] for lag_result in gc_dicts_direction_1.values()]

                gc_dicts_direction_2 = grangercausalitytests(P[[node2, node1]], MAXLAG, verbose=False)
                p_values_direction_2 = [lag_result[0]['ssr_chi2test'][1] for lag_result in gc_dicts_direction_2.values()]


                print(node1, node2)
                if np.min(p_values_direction_1) < 0.05 or np.min(p_values_direction_2) < 0.05:
                    print('causal')
                    graph.create(py2neo.Relationship(n1, "GC", n2, causal=True))
                else:
                    print('not causal')

In [19]:
granger_causality_for_neighbors(worker1_neighbors, graph)

rabbitmq queue-master
causal
rabbitmq shipping
causal
rabbitmq catalogue-db
not causal
rabbitmq catalogue
causal
rabbitmq user
causal
rabbitmq carts
causal
rabbitmq payment
causal
rabbitmq orders
causal
queue-master shipping
causal
queue-master catalogue-db
causal
queue-master catalogue
causal
queue-master user
causal
queue-master carts
causal
queue-master payment
not causal
queue-master orders
causal
shipping catalogue-db
causal
shipping catalogue
not causal
shipping user
causal
shipping carts
causal
shipping payment
not causal
shipping orders
causal
catalogue-db catalogue
not causal
catalogue-db user
causal
catalogue-db carts
causal
catalogue-db payment
not causal
catalogue-db orders
causal
catalogue user
causal
catalogue carts
not causal
catalogue payment
causal
catalogue orders
causal
user carts
causal
user payment
causal
user orders
causal
carts payment
not causal
carts orders
causal
payment orders
not causal


In [20]:
granger_causality_for_neighbors(worker2_neighbors, graph)

session-db carts-db
causal
session-db orders-db
causal
session-db user-db
causal
session-db front-end
not causal
carts-db orders-db
causal
carts-db user-db
causal
carts-db front-end
causal
orders-db user-db
causal
orders-db front-end
causal
user-db front-end
causal
